In [21]:
import json
import random

In [22]:
lang = "en"
diff = "easy"

vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(rf"vocabs\{lang}\{lang}-{diff}.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

# with open(r"vocabs\{lang}\{lang}-medium.json") as file:
#     json_data = json.load(file)
#     for d in json_data:
#         exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

26 abcdefghijklmnopqrstuvwxyz
len(vocabs_pool)=1560
len(exclusive_pool)=1560


In [23]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'xbc', 'lbc', 'abj', 'atc', 'amc', 'aby', 'abh', 'abb', 'auc', 'aqc', 'agc', 'abl', 'kbc', 'dbc', 'gbc', 'jbc', 'cbc', 'anc', 'ebc', 'axc', 'abk', 'abp', 'abo', 'abd', 'wbc', 'abg', 'awc', 'aac', 'ajc', 'abt', 'abv', 'tbc', 'abs', 'ayc', 'bbc', 'apc', 'qbc', 'nbc', 'abe', 'abq', 'fbc', 'ybc', 'acc', 'akc', 'aec', 'zbc', 'abm', 'alc', 'ubc', 'ahc', 'abn', 'avc', 'hbc', 'sbc', 'aoc', 'mbc', 'azc', 'arc', 'abu', 'abw', 'vbc', 'abx', 'abr', 'afc', 'pbc', 'abz', 'aba', 'asc', 'aic', 'ibc', 'rbc', 'abi', 'obc', 'abf', 'adc'}


In [24]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [25]:
def distance(
    from_: str, to: str, max_distance: int | None = None
) -> tuple[int, list[str]]:
    if len(from_) != len(to):
        return (-1, [])

    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [26]:
print(distance("admin", "admit"))
print(distance("line", "firm"))
print(distance("war", "him"))
print(distance("センセイ", "モンダイ"))
print(distance("caro", "mano"))

(1, ['admin', 'admit'])
(3, ['line', 'fine', 'fire', 'firm'])
(-1, [])
(-1, [])
(-1, [])


In [27]:
# easy: {"min_distance": 3, "max_distance": 4, "strict": False}
from collections import defaultdict

min_distance = 3
max_distance = 4
strict = False

starter = set()
involved = set()
counts = defaultdict(lambda: [0, 0, 0])  # vocab, pair, exclusive

pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if strict:
            difference = character_difference(v1, v2)
            flag = dist[0] > difference and dist[0] >= min_distance
        else:
            flag = dist[0] >= min_distance

        if flag:
            counts[len(v1)][1] += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                counts[len(v1)][2] += 1
                pairs.append((v1, v2))

    counts[len(v1)][0] += 1
    if (i + 1) % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i+1}/{num_vocabs}')

checking "theme"| 0/1560
checking "yield"| 156/1560
checking "pizza"| 312/1560
checking "range"| 468/1560
checking "shops"| 624/1560
checking "wants"| 780/1560
checking "spots"| 936/1560
checking "above"| 1092/1560
checking "silly"| 1248/1560
checking "such"| 1404/1560


In [28]:
total_vocab = 0
total_combinations = 0
total_pairs = 0
total_exclusive = 0

print_table = []
for k, v in counts.items():
    count_vocab = v[0]
    vocab_combinations = count_vocab * (count_vocab - 1)
    count_pair = v[1]
    count_exlusive = v[2]

    cur_pair_rate = count_pair / vocab_combinations
    cur_exclusive_rate = count_exlusive / vocab_combinations
    to_print = [
        f"len {k}",
        f"{count_vocab}",
        f"{count_pair}/{vocab_combinations}",
        f"{cur_pair_rate:.2%}",
        f"{count_exlusive}/{vocab_combinations}",
        f"{cur_exclusive_rate:.2%}",
    ]
    print_table.append(to_print)

    total_vocab += count_vocab
    total_combinations += vocab_combinations
    total_pairs += count_pair
    total_exclusive += count_exlusive

pair_rate = total_pairs / total_combinations
exclusive_rate = total_exclusive / vocab_combinations
to_print = [
    f"total",
    f"{total_vocab}",
    f"{total_pairs}/{total_combinations}",
    f"{pair_rate:.2%}",
    f"{total_exclusive}/{total_combinations}",
    f"{exclusive_rate:.2%}",
]
print_table.append(to_print)

print(f"----- pair rate & exclusive rate -----")
for items in print_table:
    print(
        f"{items[0]:<5}, #{items[1]:5}:"
        f" {items[2]:>13} = {items[3]:<7} |"
        f" {items[4]:>13} = {items[5]:<7}"
    )
print("-" * 15)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

----- pair rate & exclusive rate -----
len 5, #1166 :  3642/1358390 = 0.27%   |  3642/1358390 = 0.27%  
len 4, #394  :   3962/154842 = 2.56%   |   3962/154842 = 2.56%  
total, #1560 :  7604/1513232 = 0.50%   |  7604/1513232 = 4.91%  
---------------
starter: 615/1560 = 39.42%
participation: 650/1560 = 41.67%


In [29]:
random.seed(0)
pairs_shuffled = pairs[:]
random.shuffle(pairs_shuffled)
with open(rf"vocabs\{lang}\{lang}-pairs-{diff}.json", "w") as f:
    json.dump(pairs_shuffled, f)

for i in range(5):
    pair = pairs_shuffled[i]
    dist = distance(pair[0], pair[1])
    print(dist)

(4, ['fixes', 'fixed', 'fired', 'tired', 'tires'])
(3, ['bikes', 'likes', 'lakes', 'takes'])
(3, ['near', 'hear', 'head', 'dead'])
(3, ['leeds', 'leads', 'loads', 'loans'])
(4, ['saved', 'saves', 'sales', 'tales', 'taxes'])


In [30]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)